In [63]:
from google.colab import files
uploaded = files.upload()

Saving si-wiki-qa-original-combined.csv to si-wiki-qa-original-combined.csv


In [64]:
import pandas as pd
data=pd.read_csv("si-wiki-qa-original-combined.csv")
# data=data.drop(['context',	'question',	'answer'	],axis=1)

In [65]:
data.head()

,context,question,answer
0,රසායනික ද්‍රව්‍යරසායනික ද්‍රව්‍ය,ශ්‍රී දළදා මාළිගාව පිහිටා ඇත්තේ කොහේද ?,මහනුවර
1,ශ්‍රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම්...,ශ්‍රී දළදා මාළිගාව පිහිටා ඇති නගරය ලෝක උරුමයක්...,යුනෙස්කෝව
2,ශ්‍රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම්...,ශ්‍රී දළදා මාලිගාව මුල්වරට ගොඩනගනු ලැබුයේ කවුර...,පලමුවන විමලධර්මසූරිය
3,ශ්‍රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම්...,දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව...,ශ්‍රී වීරපරාක්‍රම නරේන්ද්‍රසිංහ
4,"තාරකා විද්‍යාව හෙවත් තාරකාවේදය යනු තාරකා, ග්‍ර...",තාරකා විද්‍යාව යනු කුමක්ද ?,පෘථිවි වායුගෝලයෙන් පරිභාහිර සංසිද්දීන් සහ අභ්‍...


In [66]:
import re
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', '', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # define punctuation
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


        preprocessed_text= ""
        for char in document:
          if char not in punctuations:
              preprocessed_text = preprocessed_text+ char
        
        return preprocessed_text

In [67]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['answer']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  lis=context+question
  return lis,context,question,answer

In [68]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=sentences.split()
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=line.split()
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab

In [69]:
lis,context,question,answer=gather_text_for_vocab(data)

In [70]:
 word2idx, idx2word, word_vocab=build_word_vocab(lis)

In [71]:
len(word2idx)

8091

In [72]:
cleaned_data=pd.DataFrame(list(zip(context,question,answer)), columns =['context_translated', 'question_translated','text_translated'])
cleaned_data

,context_translated,question_translated,text_translated
0,රසායනික ද්රව්යරසායනික ද්රව්ය,ශ්රී දළදා මාළිගාව පිහිටා ඇත්තේ කොහේද,මහනුවර
1,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,ශ්රී දළදා මාළිගාව පිහිටා ඇති නගරය ලෝක උරුමයක් ...,යුනෙස්කෝව
2,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,ශ්රී දළදා මාලිගාව මුල්වරට ගොඩනගනු ලැබුයේ කවුරු...,පලමුවන විමලධර්මසූරිය
3,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව...,ශ්රී වීරපරාක්රම නරේන්ද්රසිංහ
4,තාරකා විද්යාව හෙවත් තාරකාවේදය යනු තාරකා ග්රහලෝ...,තාරකා විද්යාව යනු කුමක්ද,පෘථිවි වායුගෝලයෙන් පරිභාහිර සංසිද්දීන් සහ අභ්ය...
...,...,...,...
528,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,එකල බ්රිතාන්ය ආණ්ඩුකාරයා වූයේ කව්ද,රොබට් බ්රවුන්රිග්
529,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,මෙම නිදහස් අරගලයෙහි නායකයන් දෙදෙනා වුයේ කව්රුන්ද,විල්බාවේ සහ කැප්පෙටිපොල දිසාවේය
530,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,නායක්කර් වංශික රාජ උරුමයක් තිබුණේ කාටද,විල්බාවේට
531,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,කැප්පෙටිපොල රෝගාතුරවීමට පෙර විරුවෝ අල්ලා ගත්තේ...,මාතලේ


In [75]:
answer_start=[]
answer_end=[]

for i in range(0,533):
  text=str(cleaned_data['context_translated'][i])
  answer=str(cleaned_data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [76]:
cleaned_data['answer_start']=answer_start
cleaned_data['answer_end']=answer_end
cleaned_data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,රසායනික ද්රව්යරසායනික ද්රව්ය,ශ්රී දළදා මාළිගාව පිහිටා ඇත්තේ කොහේද,මහනුවර,-1,6
1,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,ශ්රී දළදා මාළිගාව පිහිටා ඇති නගරය ලෝක උරුමයක් ...,යුනෙස්කෝව,604,614
2,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,ශ්රී දළදා මාලිගාව මුල්වරට ගොඩනගනු ලැබුයේ කවුරු...,පලමුවන විමලධර්මසූරිය,-1,19
3,ශ්රී දළදා මාළිගාව යනු බුදුරජාණන් වහන්සේගේ වම් ...,දැනට දක්නට ලැබෙන පැරණි දෙමහල් වැඩසිටින මාලිගාව...,ශ්රී වීරපරාක්රම නරේන්ද්රසිංහ,436,464
4,තාරකා විද්යාව හෙවත් තාරකාවේදය යනු තාරකා ග්රහලෝ...,තාරකා විද්යාව යනු කුමක්ද,පෘථිවි වායුගෝලයෙන් පරිභාහිර සංසිද්දීන් සහ අභ්ය...,-1,91
...,...,...,...,...,...
528,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,එකල බ්රිතාන්ය ආණ්ඩුකාරයා වූයේ කව්ද,රොබට් බ්රවුන්රිග්,82,99
529,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,මෙම නිදහස් අරගලයෙහි නායකයන් දෙදෙනා වුයේ කව්රුන්ද,විල්බාවේ සහ කැප්පෙටිපොල දිසාවේය,255,286
530,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,නායක්කර් වංශික රාජ උරුමයක් තිබුණේ කාටද,විල්බාවේට,294,303
531,ඌව නිදහස් අරගලය බ්රිතාන්ය පාලනයට එරෙහිව පැන නැ...,කැප්පෙටිපොල රෝගාතුරවීමට පෙර විරුවෝ අල්ලා ගත්තේ...,මාතලේ,493,498


In [77]:
cleaned_data = cleaned_data[cleaned_data.answer_start != -1]
cleaned_data=cleaned_data.reset_index()

len(cleaned_data['context_translated'])

517

In [78]:
def context_to_ids(text, word2idx):
   
    context_tokens=text.split()
    # for paragraph in stripped:
    #     tokenized=word_tokenize(paragraph)
    #     context_tokens.append(tokenized)
    
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

In [79]:
cleaned_data.to_csv('data.csv')

In [35]:
q=0
context_ids=[]
for i in range(0,len(new_df['context_translated'])):
    try:                                                        
        ids = context_to_ids(new_df['context_translated'][i] , word2idx)
        context_ids.append(ids)
        
    except Exception as e:
        context_ids.append(e)
        q=q+1
        continue

In [36]:
len(context_ids)

293